## 使用DeepSeek进行数据表的查询

In [2]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor

db_user = "root"
db_password = "123456"
#db_host = "localhost:3306"
db_host = "192.168.1.4:3306"
db_name = "project-db-001"
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")
db

In [3]:
from langchain.chat_models import ChatOpenAI
import os

# 从环境变量获取 dashscope 的 API Key
api_key = os.environ.get('DASHSCOPE_APP_KEY')

# 通过LLM => 撰写SQL
llm = ChatOpenAI(
    temperature=0.01,
    model="deepseek-v3",  
    #model = "qwen-turbo",
    # openai_api_key = "sk-9846f14a2104490b960adbf5c5b3b32e",
    # openai_api_base="https://api.deepseek.com"
    openai_api_base = "https://dashscope.aliyuncs.com/compatible-mode/v1",
    openai_api_key  = api_key
)

# 需要设置llm
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

# SQL智能体：给它目标，它自己会进行规划，最终把结果给你
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)

C:\Users\qianx\AppData\Local\Temp\ipykernel_24012\822270337.py:8: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [4]:
# Task: 描述数据表
agent_executor.run("描述与订单相关的表及其关系")

C:\Users\qianx\AppData\Local\Temp\ipykernel_24012\2182499378.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent_executor.run("描述与订单相关的表及其关系")




> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: agentinfo, beneficiaryinfo, claiminfo, crs_orders, customerinfo, employeeinfo, heros, policyinfo, productinfoI see tables that might be related to orders, particularly "crs_orders". I should examine its schema and relationships with other tables.

Action: sql_db_schema
Action Input: crs_orders, customerinfo, productinfo, policyinfo
CREATE TABLE crs_orders (
	order_time DATETIME, 
	crs_user_id BIGINT, 
	eco_main_order_id TEXT CHARACTER SET utf8mb3 COLLATE utf8mb3_general_ci, 
	channel TEXT CHARACTER SET utf8mb3 COLLATE utf8mb3_general_ci, 
	status_code TEXT CHARACTER SET utf8mb3 COLLATE utf8mb3_general_ci, 
	hotel_code TEXT CHARACTER SET utf8mb3 COLLATE utf8mb3_general_ci, 
	reserved_roomtype_code TEXT CHARACTER SET utf8mb3 COLLATE utf8mb3_general_ci, 
	actual_roomtype_code TEXT CHARACTER SET utf8mb3 COLLATE utf8mb3_general_ci, 
	rate_code TEXT CHARACTER SET utf8mb3 COLLATE utf8mb3_general_ci, 
	rooms 

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `Based on the database schema, here's the description of order-related tables and their relationships:

1. **crs_orders** - Main orders table containing:
   - Order details (order_time, eco_main_order_id, status_code)
   - Customer info (crs_user_id, age, gender)
   - Product info (productid)
   - Booking details (hotel_code, room types, rate_code, dates, revenue)
   - Location info (city, province)

2. **Relationships**:
   - The `crs_orders` table appears to connect to:
     - `customerinfo` through `crs_user_id` (though the field names don't exactly match)
     - `productinfo` through `productid` (though crs_orders.productid is text while productinfo.ProductID is bigint)
   
3. **Other potentially related tables**:
   - `policyinfo` contains insurance policy informatio

In [5]:
# 这个任务，实际上数据库中 没有HeroDetails表
agent_executor.run("描述HeroDetails表")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""agentinfo, beneficiaryinfo, claiminfo, crs_orders, customerinfo, employeeinfo, heros, policyinfo, productinfoI need to check if there's a table called "HeroDetails" since the user asked about it, but from the list of tables I see "heros" which might be related.
Action: sql_db_schema
Action Input: "heros"
CREATE TABLE heros (
	id INTEGER NOT NULL AUTO_INCREMENT, 
	name VARCHAR(255) CHARACTER SET utf8mb3 COLLATE utf8mb3_general_ci NOT NULL, 
	hp_max FLOAT, 
	hp_growth FLOAT, 
	hp_start FLOAT, 
	mp_max FLOAT, 
	mp_growth FLOAT, 
	mp_start FLOAT, 
	attack_max FLOAT, 
	attack_growth FLOAT, 
	attack_start FLOAT, 
	defense_max FLOAT, 
	defense_growth FLOAT, 
	defense_start FLOAT, 
	hp_5s_max FLOAT, 
	hp_5s_growth FLOAT, 
	hp_5s_start FLOAT, 
	mp_5s_max FLOAT, 
	mp_5s_growth FLOAT, 
	mp_5s_start FLOAT, 
	attack_speed_max FLOAT, 
	attack_range VARCHAR(255) CHARACTER SET utf8mb3 COLLATE utf8mb3_general_ci, 
	

'The "heros" table (which appears to be the closest match to "HeroDetails") contains detailed information about heroes including their attributes and growth statistics. The table has columns for ID, name, various health points (HP) metrics (max, growth, start), magic points (MP) metrics, attack and defense metrics, regeneration rates, attack speed, attack range, main and assist roles, and birthdate. Here are some example columns: id, name, hp_max, hp_growth, attack_max, defense_max, role_main, role_assist, and birthdate. The table stores game character statistics with growth values over time.'

In [6]:
agent_executor.run("描述Hero表")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""agentinfo, beneficiaryinfo, claiminfo, crs_orders, customerinfo, employeeinfo, heros, policyinfo, productinfoI see there's a table called "heros" (note the plural form) that seems relevant to the question. I should query its schema to understand its structure.

Action: sql_db_schema
Action Input: heros
CREATE TABLE heros (
	id INTEGER NOT NULL AUTO_INCREMENT, 
	name VARCHAR(255) CHARACTER SET utf8mb3 COLLATE utf8mb3_general_ci NOT NULL, 
	hp_max FLOAT, 
	hp_growth FLOAT, 
	hp_start FLOAT, 
	mp_max FLOAT, 
	mp_growth FLOAT, 
	mp_start FLOAT, 
	attack_max FLOAT, 
	attack_growth FLOAT, 
	attack_start FLOAT, 
	defense_max FLOAT, 
	defense_growth FLOAT, 
	defense_start FLOAT, 
	hp_5s_max FLOAT, 
	hp_5s_growth FLOAT, 
	hp_5s_start FLOAT, 
	mp_5s_max FLOAT, 
	mp_5s_growth FLOAT, 
	mp_5s_start FLOAT, 
	attack_speed_max FLOAT, 
	attack_range VARCHAR(255) CHARACTER SET utf8mb3 COLLATE utf8mb3_general_ci, 
	ro

'Hero表（实际表名为heros）是一个存储英雄信息的表，包含以下字段：\n1. id: 自增主键\n2. name: 英雄名称\n3. hp_max: 最大生命值\n4. hp_growth: 生命值成长\n5. hp_start: 初始生命值\n6. mp_max: 最大法力值\n7. mp_growth: 法力值成长\n8. mp_start: 初始法力值\n9. attack_max: 最大攻击力\n10. attack_growth: 攻击力成长\n11. attack_start: 初始攻击力\n12. defense_max: 最大防御力\n13. defense_growth: 防御力成长\n14. defense_start: 初始防御力\n15. hp_5s_max: 每5秒最大回血\n16. hp_5s_growth: 回血成长\n17. hp_5s_start: 初始回血\n18. mp_5s_max: 每5秒最大回蓝\n19. mp_5s_growth: 回蓝成长\n20. mp_5s_start: 初始回蓝\n21. attack_speed_max: 最大攻速\n22. attack_range: 攻击范围（近战/远程）\n23. role_main: 主要角色（如坦克、战士等）\n24. role_assist: 辅助角色\n25. birthdate: 英雄创建日期\n\n示例数据包括夏侯惇、钟无艳等英雄的详细属性信息。'

In [7]:
agent_executor.run("找出英雄攻击力最高的前5个英雄")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: agentinfo, beneficiaryinfo, claiminfo, crs_orders, customerinfo, employeeinfo, heros, policyinfo, productinfoI see there's a table called "heros" which is likely to contain information about heroes. I should check its schema to find the relevant columns for attack power.

Action: sql_db_schema
Action Input: heros
CREATE TABLE heros (
	id INTEGER NOT NULL AUTO_INCREMENT, 
	name VARCHAR(255) CHARACTER SET utf8mb3 COLLATE utf8mb3_general_ci NOT NULL, 
	hp_max FLOAT, 
	hp_growth FLOAT, 
	hp_start FLOAT, 
	mp_max FLOAT, 
	mp_growth FLOAT, 
	mp_start FLOAT, 
	attack_max FLOAT, 
	attack_growth FLOAT, 
	attack_start FLOAT, 
	defense_max FLOAT, 
	defense_growth FLOAT, 
	defense_start FLOAT, 
	hp_5s_max FLOAT, 
	hp_5s_growth FLOAT, 
	hp_5s_start FLOAT, 
	mp_5s_max FLOAT, 
	mp_5s_growth FLOAT, 
	mp_5s_start FLOAT, 
	attack_speed_max FLOAT, 
	attack_range VARCHAR(255) CHARACTER SET utf8mb3 COLLATE utf8mb3_general

'攻击力最高的前5个英雄是：\n1. 阿轲 (攻击力: 427)\n2. 孙尚香 (攻击力: 411)\n3. 百里守约 (攻击力: 410)\n4. 虞姬 (攻击力: 407)\n5. 黄忠 (攻击力: 403)'